<h1><font color='green'>Assignment CNN on CIFAR

In [1]:
#Importing librarries

import tensorflow as tf
import numpy as np
from keras import backend as k
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator


C:\Users\RJ\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\RJ\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\RJ\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\RJ\AppData\Roaming\Python\Python37\site-packages\ten

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

k.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 200
l = 10
num_filter = 42
compression = 0.5
dropout_rate = 0

In [4]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [5]:
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 10)
(10000, 10)


In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

<h2><font color='green'>Image Augmentaiton

In [0]:
# https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation
#for Image data augmentation on trainig data

datagen_tr = ImageDataGenerator(featurewise_center=False, samplewise_center=False, 
                                featurewise_std_normalization=False, samplewise_std_normalization=False,
                                zca_whitening=False, zca_epsilon=1e-06,
                                rotation_range=0, width_shift_range=0.1, height_shift_range=0.1,
                                shear_range=0., zoom_range=0., channel_shift_range=0., 
                                fill_mode='nearest', cval=0., horizontal_flip=True,vertical_flip=False, 
                                rescale=None, preprocessing_function=None, data_format=None, validation_split=0.0)

#for Image data augmentation on testing data
datagen_te = ImageDataGenerator(rescale=1./255)

In [0]:
#Image data augmentation on train and test data
datagen_tr.fit(X_train)
datagen_te.fit(X_test)

<h2><font color='green'>Model

In [0]:
# Dense Block
def denseblock(input, num_filter = 42, dropout_rate = 0):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Block
def transition(input, num_filter = 42, dropout_rate = 0):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output block
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)

    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    conv2 = layers.Conv2D(num_classes, (3,3), use_bias=False, padding='same')(AvgPooling)
    globalmax = layers.GlobalMaxPooling2D()(conv2)
    
    output = layers.Activation('softmax')(globalmax)

    return output

In [10]:
num_filter = 42
dropout_rate = 0
l = 10

input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 42)   1134        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 42)   168         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 42)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [13]:
history = model.fit_generator(datagen_tr.flow(X_train, y_train, batch_size=batch_size),
                              steps_per_epoch=len(X_train) / batch_size, epochs = epochs,
                              validation_data=(X_test, y_test))

Epoch 1/200
391/390 [==============================] - 212s 542ms/step - loss: 1.4269 - acc: 0.4776 - val_loss: 1.8102 - val_acc: 0.3677
Epoch 2/200
391/390 [==============================] - 192s 492ms/step - loss: 0.9997 - acc: 0.6421 - val_loss: 1.1900 - val_acc: 0.6078
Epoch 3/200
391/390 [==============================] - 193s 493ms/step - loss: 0.7976 - acc: 0.7175 - val_loss: 0.8601 - val_acc: 0.7081
Epoch 4/200
391/390 [==============================] - 192s 492ms/step - loss: 0.6813 - acc: 0.7615 - val_loss: 0.7056 - val_acc: 0.7577
Epoch 5/200
391/390 [==============================] - 192s 492ms/step - loss: 0.6095 - acc: 0.7874 - val_loss: 1.0214 - val_acc: 0.6759
Epoch 6/200
391/390 [==============================] - 192s 492ms/step - loss: 0.5540 - acc: 0.8066 - val_loss: 0.8931 - val_acc: 0.7146
Epoch 7/200
391/390 [==============================] - 193s 492ms/step - loss: 0.5085 - acc: 0.8225 - val_loss: 0.6561 - val_acc: 0.7796
Epoch 8/200
391/390 [====================

In [16]:
# Test score
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 12s 1ms/sample - loss: 0.5320 - acc: 0.9006
Test loss: 0.5319637671692297
Test accuracy: 0.9006


<h2><font color='orange'>Conclusion

In [17]:
from prettytable import PrettyTable

x = PrettyTable()

x.field_names = ["Model", "Training accuracy", "Testing accuracy"]
                 
x.add_row(["DenseNet", "99.27%", "99.06%"])
       
print(x)

+----------+-------------------+------------------+
|  Model   | Training accuracy | Testing accuracy |
+----------+-------------------+------------------+
| DenseNet |       99.27%      |      99.06%      |
+----------+-------------------+------------------+
